In [1]:
import pandas as pd

In [2]:
import nettoyage_lib as nl

In [3]:
donnees_brutes = pd.read_csv("https://raw.githubusercontent.com/VPerrollaz/immobilier/master/donnees/data.tsv", sep="\t")

In [4]:
donnees_nettoyees = (
    donnees_brutes
    .pipe(nl.conversion_types)  
    .pipe(nl.suppression_annonces_redondantes)
    .pipe(nl.suppression_colonne_id)
    .pipe(nl.numerise_les_colonnes)
    .pipe(nl.supprime_partiellement_na)
)

In [41]:
donnees_nettoyees.head()

,Genre,Neuf,Surface,Pieces,Quartier,Prix
0,0,0,90.00,3,cathédrale,374400
1,0,0,146.27,5,sud,499200
2,0,0,110.00,5,prébendes,499200
3,1,0,132.00,6,prébendes,508000
4,1,0,185.00,7,strasbourg,676000


In [45]:
donnees_nettoyees.dtypes

Genre         int64
Neuf          Int64
Surface     float64
Pieces        Int64
Quartier     string
Prix          Int64
dtype: object

In [47]:
donnees_nettoyees.Quartier = donnees_nettoyees.Quartier.astype("category")
donnees_nettoyees.dtypes

Genre          int64
Neuf           Int64
Surface      float64
Pieces         Int64
Quartier    category
Prix           Int64
dtype: object

**ATTENTION** cela devrait être une étape supplémentaire dans le nettoyage ci-dessus.

In [11]:
# Les différents modèles considérés.
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

In [12]:
# Pour évaluer les différences modèles
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV

In [7]:
# Pour transformer les variables en numérique quand elles sont catégorielles
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [9]:
# Pour normaliser les colonnes (intérêt principalement numérique pour faciliter la vie des solveurs)
from sklearn.preprocessing import MinMaxScaler, Normalizer, RobustScaler, StandardScaler

In [13]:
# Pour gérer les données manquantes
from sklearn.impute import SimpleImputer

In [15]:
# Pour assemble des pipelines
from sklearn.pipeline import Pipeline, FeatureUnion

In [42]:
# Pour gérer les variables indépendamment
from sklearn.compose import ColumnTransformer

# Gérer la colonne quartier

- Utiliser `LabelEncoder`.
- Utiliser `OneHotEncoder`.
- Gérer les données manquantes.

In [25]:
quartier = pd.DataFrame(donnees_nettoyees[["Quartier"]], dtype="category")
type(quartier)

pandas.core.frame.DataFrame

In [26]:
quartier.dtypes

Quartier    category
dtype: object

In [27]:
si = SimpleImputer(strategy="constant", fill_value="Donnee Manquante")
si.fit(quartier)

SimpleImputer(fill_value='Donnee Manquante', strategy='constant')

In [33]:
resultat_intermediaire = si.transform(quartier).reshape(-1)

In [34]:
le = LabelEncoder()
resultat = le.fit(resultat_intermediaire).transform(resultat_intermediaire)


**ATTENTION** le label encoder est plutôt destinée à la transformation de vecteur (plutôt donc le y) car on induit des structures d'ordre et de distance qui peuvent biaiser la suite de l'entrainement.

In [39]:
gestion_quartier = Pipeline([
    ("GestionNaN", SimpleImputer(strategy="constant", fill_value="Donnee Manquante")),
    ("Numérisation", OneHotEncoder(sparse=False))
])

In [40]:
gestion_quartier.fit_transform(quartier)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

**BILAN** On a crée une catégorie spécifique pour les données manquantes on pourrait à la suite de ce pipeline appliquée des transformations plus sophistiquée:
- Les lignes où on avait une donnée manquante on insére la moyenne sur les autres colonnes.
- On filtre les lignes qui ont une donnée manquante.
- On laisse tel quel.

# Intégration au reste des colonnes.

Regarder la documentation de `FeatureUnion` et `ColumnTransformer` et créer un pipeline qui fait passer du DataFrame au X que l'on pourra injecter dans les modèles.

In [53]:
ct = ColumnTransformer([
    (
        "Gestion Quartier", 
        Pipeline([
            ("GestionNaN", SimpleImputer(strategy="constant", fill_value="Donnee Manquante")),
            ("Numérisation", OneHotEncoder(sparse=False))
    
        ]), 
        ["Quartier"]
    ),
    (
        "Colonnes déja numériques",
        MinMaxScaler(),
        ["Genre", "Neuf", "Surface", "Pieces"]
    )
])

In [59]:
X = ct.fit_transform(donnees_nettoyees)
X.shape

(1625, 31)

In [60]:
X.min(axis=0), X.max(axis=0)

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [61]:
y = donnees_nettoyees.Prix.values
y.shape

(1625,)

# Reste la partie apprentissage

- On pourra recommencer la méthodologie de la semaine dernière.
- On peut aussi utiliser les `RandomizedSearchCV` et `GridSearchCV`.
- On peut inclure le préprocessing dans la partie crossvalidation si on a plusieurs choix possible.

**REMARQUES** 
- la jonction entre pandas et sklearn et décidée (plus ou moins) suivant le principe que si on différents choix possibles pour une même étape, on choisira parmi ces possibilités par crossvalidation et on se placera donc dans scikitlearn.
- On pourrait créer un objet sklearn sur mesure (implémentant fit et transform) pour inclure la partie pandas dans le pipeline sklearn.